**Transfer learning**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Fashion MNIST
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
X_train_resized = tf.image.resize(X_train, (96, 96))
X_test_resized = tf.image.resize(X_test, (96, 96))

# Preprocess for MobileNetV2
# The images are already in RGB format, so no need for grayscale_to_rgb conversion.
X_train_rgb = preprocess_input(X_train_resized.numpy())
X_test_rgb = preprocess_input(X_test_resized.numpy())

# One-hot encode labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [ ]:
base_model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), # Pool spatial features into a vector
    layers.Dense(128, activation='relu'), # Fully connected layer to learn new patterns
    layers.Dense(10, activation='softmax') # Output layer with 10 classes (Fashion MNIST)
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_rgb, y_train_cat, validation_split=0.2, epochs=5, batch_size=64)

In [ ]:
est_loss, test_acc = model.evaluate(X_test_rgb, y_test_cat)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Predictions
y_pred_probs = model.predict(X_test_rgb)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=class_names)
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


In [ ]:
from tensorflow.keras import optimizers


In [ ]:
base_model.trainable = True

# Freeze all layers except the last N (e.g., last 20 layers)
fine_tune_at = len(base_model.layers) - 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training (fine-tuning)
fine_tune_epochs = 5
total_epochs = 3 + fine_tune_epochs  # if trained earlier 3 epochs

history_fine = model.fit(
    X_train_rgb, y_train_cat,
    validation_split=0.2,
    epochs=total_epochs,
    initial_epoch=3,
    batch_size=64
)



In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(X_test_rgb, y_test_cat)
print(f"Test Accuracy after fine-tuning: {test_acc:.4f}")